In [1]:
import pandas as pd

import datetime as dt
from datetime import timedelta
import numpy as np

from config import pwd
from sqlalchemy import create_engine,inspect,extract
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [2]:
Accidents_df = pd.read_csv("US_Accidents_Dec20_updated.csv")

In [12]:
Accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1516064 entries, 0 to 1516063
Data columns (total 47 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   ID                     1516064 non-null  object 
 1   Severity               1516064 non-null  int64  
 2   Start_Time             1516064 non-null  object 
 3   End_Time               1516064 non-null  object 
 4   Start_Lat              1516064 non-null  float64
 5   Start_Lng              1516064 non-null  float64
 6   End_Lat                1516064 non-null  float64
 7   End_Lng                1516064 non-null  float64
 8   Distance(mi)           1516064 non-null  float64
 9   Description            1516064 non-null  object 
 10  Number                 469969 non-null   float64
 11  Street                 1516064 non-null  object 
 12  Side                   1516064 non-null  object 
 13  City                   1515981 non-null  object 
 14  County            

In [3]:
Newdata_df = Accidents_df[['ID','Severity','Start_Time','End_Time','Start_Lat','Start_Lng','Street','City','State',
              'Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station',
              'Stop','Traffic_Calming','Traffic_Signal','Turning_Loop','Weather_Condition', 'Wind_Speed(mph)', 'Precipitation(in)']].copy()

In [4]:
Newdata_df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Street,City,State,Amenity,...,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Weather_Condition,Wind_Speed(mph),Precipitation(in)
0,A-2716600,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.10891,-83.09286,Outerbelt E,Dublin,OH,False,...,False,False,False,False,False,False,False,Light Rain,10.4,0.00
1,A-2716601,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.86542,-84.06280,I-70 E,Dayton,OH,False,...,False,False,False,False,False,False,False,Light Rain,NaN,0.02
2,A-2716602,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10266,-84.52468,I-75 S,Cincinnati,OH,False,...,False,False,False,False,False,False,False,Overcast,NaN,0.02
3,A-2716603,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10148,-84.52341,US-50 E,Cincinnati,OH,False,...,False,False,False,False,False,False,False,Overcast,NaN,0.02
4,A-2716604,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.06213,-81.53784,I-77 N,Akron,OH,False,...,False,False,False,False,False,False,False,Overcast,NaN,NaN


In [5]:
Weather_df = Newdata_df[['ID','Severity','Weather_Condition','City','State','Start_Time','Wind_Speed(mph)','Precipitation(in)']].copy()

In [6]:
Weather_df.rename(columns={"ID":"Accident_id",
                            "Precipitation(in)":"Precipitation",
                          "Wind_Speed(mph)":"Wind_Speed",
                          "Year":"StartDate",
                          "AccYear":"Year"},inplace=True)

In [7]:
Newdata_df.rename(columns={"ID":"Accident_id",
                            "Precipitation(in)":"Precipitation",
                          "Wind_Speed(mph)":"Wind_Speed",
                          "Year":"StartDate"
                          },inplace=True)

In [8]:
Weather_df.head()

,Accident_id,Severity,Weather_Condition,City,State,Start_Time,Wind_Speed,Precipitation
0,A-2716600,3,Light Rain,Dublin,OH,2016-02-08 00:37:08,10.4,0.00
1,A-2716601,2,Light Rain,Dayton,OH,2016-02-08 05:56:20,NaN,0.02
2,A-2716602,2,Overcast,Cincinnati,OH,2016-02-08 06:15:39,NaN,0.02
3,A-2716603,2,Overcast,Cincinnati,OH,2016-02-08 06:15:39,NaN,0.02
4,A-2716604,2,Overcast,Akron,OH,2016-02-08 06:51:45,NaN,NaN


In [9]:
Newdata_df['Time']= pd.to_datetime(Newdata_df['Start_Time'],errors='coerce').dt.time

In [10]:
Newdata_df['StartDate'] = pd.to_datetime(Newdata_df['Start_Time'],errors='coerce').dt.date

In [11]:
Newdata_df['Start_year'] = pd.to_datetime(Newdata_df['StartDate'],errors='ignore').dt.year

In [12]:
Newdata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1516064 entries, 0 to 1516063
Data columns (total 28 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Accident_id        1516064 non-null  object 
 1   Severity           1516064 non-null  int64  
 2   Start_Time         1516064 non-null  object 
 3   End_Time           1516064 non-null  object 
 4   Start_Lat          1516064 non-null  float64
 5   Start_Lng          1516064 non-null  float64
 6   Street             1516064 non-null  object 
 7   City               1515981 non-null  object 
 8   State              1516064 non-null  object 
 9   Amenity            1516064 non-null  bool   
 10  Bump               1516064 non-null  bool   
 11  Crossing           1516064 non-null  bool   
 12  Give_Way           1516064 non-null  bool   
 13  Junction           1516064 non-null  bool   
 14  No_Exit            1516064 non-null  bool   
 15  Railway            1516064 non-n

In [13]:
NewFile_df = Newdata_df.loc[(Newdata_df['Start_year']== 2019.0)|(Newdata_df['Start_year']== 2020.0)]
NewFile_df

,Accident_id,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Street,City,State,Amenity,...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Weather_Condition,Wind_Speed,Precipitation,Time,StartDate,Start_year
237331,A-2953938,2,2020-11-01 22:33:04,2020-11-02 00:11:16,39.594265,-76.671599,I-83 S,Parkton,MD,False,...,False,False,False,False,Partly Cloudy / Windy,22.0,0.0,22:33:04,2020-11-01,2020
237332,A-2953939,2,2020-12-14 14:19:00,2020-12-14 17:06:29,37.701913,-121.910487,I-580 W,Dublin,CA,False,...,False,False,False,False,Fair,3.0,0.0,14:19:00,2020-12-14,2020
237333,A-2953940,2,2020-12-31 19:20:00,2020-12-31 20:58:08,37.006813,-121.558409,S Valley Fwy S,Gilroy,CA,False,...,False,False,False,False,Fair,9.0,NaN,19:20:00,2020-12-31,2020
237334,A-2953941,2,2020-10-15 23:03:55,2020-10-16 00:20:53,39.446521,-76.330792,I-95 N,Joppa,MD,False,...,False,False,False,False,Cloudy,7.0,0.0,23:03:55,2020-10-15,2020
237335,A-2953942,2,2020-12-01 14:15:55,2020-12-01 16:15:42,39.221910,-76.801697,MD-100 W,Ellicott City,MD,False,...,False,False,False,False,Mostly Cloudy,18.0,0.0,14:15:55,2020-12-01,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516059,A-4239402,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.002480,-117.379360,Pomona Fwy E,Riverside,CA,False,...,False,False,False,False,Fair,13.0,0.0,18:03:25,2019-08-23,2019
1516060,A-4239403,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.766960,-117.148060,I-8 W,San Diego,CA,False,...,False,False,False,False,Fair,6.0,0.0,19:11:30,2019-08-23,2019
1516061,A-4239404,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.775450,-117.847790,Garden Grove Fwy,Orange,CA,False,...,False,False,False,False,Partly Cloudy,10.0,0.0,19:00:21,2019-08-23,2019
1516062,A-4239405,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.992460,-118.403020,San Diego Fwy S,Culver City,CA,False,...,False,False,False,False,Fair,8.0,0.0,19:00:21,2019-08-23,2019


In [14]:
SelectCities_df = NewFile_df.loc[(NewFile_df['City']== 'Dallas') & (NewFile_df['State']== 'TX') |(NewFile_df['City']== 'New York') & (NewFile_df['State']== 'NY')|(NewFile_df['City']== 'San Diego') & (NewFile_df['State']== 'CA')|(NewFile_df['City']== 'Los Angeles') & (NewFile_df['State']== 'CA')]
SelectCities_df

,Accident_id,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Street,City,State,Amenity,...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Weather_Condition,Wind_Speed,Precipitation,Time,StartDate,Start_year
237383,A-2953990,2,2020-11-28 01:33:00,2020-11-28 03:13:16,33.963490,-118.370694,S La Cienega Blvd,Los Angeles,CA,False,...,True,False,True,False,Fair,0.0,0.0,01:33:00,2020-11-28,2020
237386,A-2953993,2,2020-11-07 12:05:00,2020-11-07 16:03:00,32.917963,-96.856461,I-635 W,Dallas,TX,False,...,False,False,False,False,Fair,7.0,0.0,12:05:00,2020-11-07,2020
237387,A-2953994,2,2020-11-03 17:10:08,2020-11-03 18:00:58,32.921262,-96.848712,I-635 W,Dallas,TX,False,...,False,False,False,False,Fair,12.0,0.0,17:10:08,2020-11-03,2020
237403,A-2954010,2,2020-11-10 02:32:00,2020-11-10 04:46:20,34.152964,-118.278776,CA-134 E,Los Angeles,CA,False,...,False,False,False,False,Fair,0.0,0.0,02:32:00,2020-11-10,2020
237454,A-2954063,2,2020-12-23 05:22:00,2020-12-23 11:31:41,34.071218,-118.235087,Stadium Way,Los Angeles,CA,False,...,False,False,False,False,Haze,3.0,0.0,05:22:00,2020-12-23,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516034,A-4239377,2,2019-08-23 13:42:50,2019-08-23 14:10:06,34.070610,-118.263910,Hollywood Fwy S,Los Angeles,CA,False,...,False,False,False,False,Fair,0.0,0.0,13:42:50,2019-08-23,2019
1516044,A-4239387,2,2019-08-23 15:33:46,2019-08-23 16:03:05,32.773510,-117.161180,CA-163 S,San Diego,CA,False,...,False,False,False,False,Fair,7.0,0.0,15:33:46,2019-08-23,2019
1516046,A-4239389,2,2019-08-23 15:45:43,2019-08-23 16:14:31,34.043650,-118.443730,I-405 N,Los Angeles,CA,False,...,False,False,False,False,Fair,8.0,0.0,15:45:43,2019-08-23,2019
1516047,A-4239390,2,2019-08-23 16:43:59,2019-08-23 17:12:34,32.729250,-117.106810,I-805 S,San Diego,CA,False,...,False,False,False,False,Fair,7.0,0.0,16:43:59,2019-08-23,2019


In [15]:
Weather_df['Year'] = pd.to_datetime(Weather_df['Start_Time'],errors = 'coerce').dt.date

In [16]:
Weather_df['Year'] = Weather_df['Year'].astype('datetime64')

In [17]:
Weather_df['AccYear'] = pd.to_datetime(Weather_df['Year'],errors='ignore').dt.year
Weather_df

,Accident_id,Severity,Weather_Condition,City,State,Start_Time,Wind_Speed,Precipitation,Year,AccYear
0,A-2716600,3,Light Rain,Dublin,OH,2016-02-08 00:37:08,10.4,0.00,2016-02-08,2016
1,A-2716601,2,Light Rain,Dayton,OH,2016-02-08 05:56:20,NaN,0.02,2016-02-08,2016
2,A-2716602,2,Overcast,Cincinnati,OH,2016-02-08 06:15:39,NaN,0.02,2016-02-08,2016
3,A-2716603,2,Overcast,Cincinnati,OH,2016-02-08 06:15:39,NaN,0.02,2016-02-08,2016
4,A-2716604,2,Overcast,Akron,OH,2016-02-08 06:51:45,NaN,NaN,2016-02-08,2016
...,...,...,...,...,...,...,...,...,...,...
1516059,A-4239402,2,Fair,Riverside,CA,2019-08-23 18:03:25,13.0,0.00,2019-08-23,2019
1516060,A-4239403,2,Fair,San Diego,CA,2019-08-23 19:11:30,6.0,0.00,2019-08-23,2019
1516061,A-4239404,2,Partly Cloudy,Orange,CA,2019-08-23 19:00:21,10.0,0.00,2019-08-23,2019
1516062,A-4239405,2,Fair,Culver City,CA,2019-08-23 19:00:21,8.0,0.00,2019-08-23,2019


In [18]:
cityweather_df = Weather_df.loc[(Weather_df['City']== 'Dallas') & (Weather_df['State']== 'TX') |(Weather_df['City']== 'New York') & (Weather_df['State']== 'NY')|(Weather_df['City']== 'San Diego') & (Weather_df['State']== 'CA')|(Weather_df['City']== 'Los Angeles') & (Weather_df['State']== 'CA')]
cityweather_df

,Accident_id,Severity,Weather_Condition,City,State,Start_Time,Wind_Speed,Precipitation,Year,AccYear
5360,A-2721960,2,Clear,Los Angeles,CA,2016-03-22 19:36:44,9.2,NaN,2016-03-22,2016
5363,A-2721963,2,Clear,San Diego,CA,2016-03-22 20:15:13,8.1,NaN,2016-03-22,2016
5364,A-2721964,2,Clear,Los Angeles,CA,2016-03-22 20:59:43,11.5,NaN,2016-03-22,2016
5379,A-2721979,3,Clear,Los Angeles,CA,2016-03-23 07:59:47,NaN,NaN,2016-03-23,2016
5385,A-2721985,2,Clear,San Diego,CA,2016-03-23 09:10:36,NaN,NaN,2016-03-23,2016
...,...,...,...,...,...,...,...,...,...,...
1516034,A-4239377,2,Fair,Los Angeles,CA,2019-08-23 13:42:50,0.0,0.0,2019-08-23,2019
1516044,A-4239387,2,Fair,San Diego,CA,2019-08-23 15:33:46,7.0,0.0,2019-08-23,2019
1516046,A-4239389,2,Fair,Los Angeles,CA,2019-08-23 15:45:43,8.0,0.0,2019-08-23,2019
1516047,A-4239390,2,Fair,San Diego,CA,2019-08-23 16:43:59,7.0,0.0,2019-08-23,2019


In [19]:
AccWeather_df = cityweather_df.loc[(cityweather_df['AccYear']==2019.0)|(cityweather_df['AccYear']== 2020.0)]
AccWeather_df

,Accident_id,Severity,Weather_Condition,City,State,Start_Time,Wind_Speed,Precipitation,Year,AccYear
237383,A-2953990,2,Fair,Los Angeles,CA,2020-11-28 01:33:00,0.0,0.0,2020-11-28,2020
237386,A-2953993,2,Fair,Dallas,TX,2020-11-07 12:05:00,7.0,0.0,2020-11-07,2020
237387,A-2953994,2,Fair,Dallas,TX,2020-11-03 17:10:08,12.0,0.0,2020-11-03,2020
237403,A-2954010,2,Fair,Los Angeles,CA,2020-11-10 02:32:00,0.0,0.0,2020-11-10,2020
237454,A-2954063,2,Haze,Los Angeles,CA,2020-12-23 05:22:00,3.0,0.0,2020-12-23,2020
...,...,...,...,...,...,...,...,...,...,...
1516034,A-4239377,2,Fair,Los Angeles,CA,2019-08-23 13:42:50,0.0,0.0,2019-08-23,2019
1516044,A-4239387,2,Fair,San Diego,CA,2019-08-23 15:33:46,7.0,0.0,2019-08-23,2019
1516046,A-4239389,2,Fair,Los Angeles,CA,2019-08-23 15:45:43,8.0,0.0,2019-08-23,2019
1516047,A-4239390,2,Fair,San Diego,CA,2019-08-23 16:43:59,7.0,0.0,2019-08-23,2019


In [20]:
SelectCities_df.to_csv("USAccidents_Clean.csv",index=False)

In [21]:
AccWeather_df.to_csv("WeatherData.csv",index=False)

In [22]:
from sqlalchemy import Column, Integer, String, Float,BigInteger,Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import func
Base=declarative_base()

In [23]:
class USAccidents(Base):
    __tablename__='us_accidents'
    id = Column(Integer, primary_key=True)
    Accident_id= Column(String)
    Severity= Column(Integer)
    Start_Time=Column(String)
    End_Time= Column(String)
    Start_Lat= Column(Float)
    Start_Lng= Column(Float)
    Street=Column(String)
    City =Column(String)
    State= Column(String)
    Weather_Condition=Column(String)
    
    Amenity= Column(Boolean)
    Bump=Column(Boolean) 
    Crossing= Column(Boolean)
    Give_Way=Column(Boolean)
    Junction=Column(Boolean) 
    No_Exit =Column(Boolean)
    Railway =Column(Boolean)
    Roundabout=Column(Boolean) 
    Station =Column(Boolean)
    Stop =Column(Boolean)
    Traffic_Calming=Column(Boolean)
    Traffic_Signal= Column(Boolean)
    Turning_Loop =Column(Boolean)
    Time= Column(String)
    StartDate=Column(String)
    Year= Column(Integer)

In [24]:
class USWeather(Base):
    __tablename__='us_weather'
    id = Column(Integer, primary_key=True)
    Accident_id = Column(String)
    Severity=Column(Integer)
    Weather_Condition =Column(String)         
    Precipitation= Column(Float)             
    Wind_Speed = Column(Float)               
    Start_Time = Column(String)         
    City = Column(String)                     
    State = Column(String)                    
    StartDate = Column(String)         
    Year =Column(Integer)
    AccYear = Column(String)

In [25]:
engine = create_engine('sqlite:///us_accidents.db', echo=False)
conn=engine.connect()
Base.metadata.create_all(engine)

In [26]:
conn.execute("DROP TABLE us_accidents ")
conn.execute("DROP TABLE us_weather ")

In [27]:
SelectCities_df.to_sql('us_accidents', con=engine,if_exists="append",index=False)
Base.metadata.create_all(engine)

In [28]:
AccWeather_df.to_sql('us_weather', con=engine,if_exists="append",index=False)
Base.metadata.create_all(engine)

In [29]:
engine = create_engine('sqlite:///us_accidents.db', echo=False)
conn=engine.connect()